# Import libraries

In [ ]:
import os
import shutil
import numpy as np
from sklearn.model_selection import train_test_split
%matplotlib inline
import scipy.io # To use the '.mat' files
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import librosa.display
from scipy.signal import butter, filtfilt
import librosa
import cv2
from matplotlib.colors import Normalize
from matplotlib import cm
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

# Mount Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Prepare datasets for ResNet model

In [ ]:
train_dir = '/content/drive/MyDrive/New_MF_data/New_Class/Reduced_Dataset_DNN/train'
validation_dir = '/content/drive/MyDrive/New_MF_data/New_Class/Reduced_Dataset_DNN/validation'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=128,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=128,
    class_mode='categorical'
)


Found 5511 images belonging to 4 classes.
Found 1838 images belonging to 4 classes.


# Define ResNet model

In [ ]:
# display, transform, read, split ...
import numpy as np
import cv2 as cv
import os
import matplotlib.pyplot as plt
# tensorflow
import tensorflow.keras as keras
import tensorflow as tf
# image processing
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
# model / neural network
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import KFold
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout


In [ ]:
def create_datagen():
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        preprocessing_function=tf.keras.applications.resnet50.preprocess_input)
    return train_datagen

def load_data(directory, img_height, img_width, batch_size):
    datagen = create_datagen()
    generator = datagen.flow_from_directory(
        directory,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True)
    return generator


In [ ]:
def build_model(num_classes):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    # Freeze the layers of the base model
    for layer in base_model.layers:
        layer.trainable = False

    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
model = build_model(4)

94765736/94765736 [==============================] - 4s 0us/step


# Implement 5 cross-validation for train and evaluate ResNet model (Batch size epochs)

In [ ]:
def train_and_evaluate_model(directory, num_splits=5):
    img_height, img_width = 224, 224
    batch_size = 128
    num_classes = 4
    epochs=30

    kf = KFold(n_splits=num_splits, shuffle=True, random_state=42)
    fold_var = 1

    results = []
    for train_index, val_index in kf.split(np.zeros(3000), np.zeros(3000)):  # Adjust size based on your dataset
        training_data = '/content/drive/MyDrive/New_MF_data/New_Class/Reduced_Dataset_DNN/train'  # Adjust path
        validation_data = '/content/drive/MyDrive/New_MF_data/New_Class/Reduced_Dataset_DNN/validation'
        train_generator = load_data(training_data, img_height, img_width, batch_size)
        validation_generator = load_data(validation_data, img_height, img_width, batch_size)

        model = build_model(num_classes)

        # Fit data to model
        history = model.fit(train_generator, steps_per_epoch=train_generator.samples // batch_size,
                            validation_data=validation_generator,
                            validation_steps=validation_generator.samples // batch_size,
                            epochs=30, verbose=1)

        results.append(history)

        fold_var += 1

    return model, results


# Run the training and validation datasets on ResNet model

In [ ]:
directory = '/content/drive/MyDrive/New_MF_data/New_Class/Reduced_Dataset_DNN'
model, results=train_and_evaluate_model(directory)


Found 5511 images belonging to 4 classes.
Found 1838 images belonging to 4 classes.


Epoch 1/30
30/43 [===================>..........] - ETA: 13:27 - loss: 1.6002 - accuracy: 0.3315

# Save the model

In [ ]:
# Assuming 'model' is your model instance
model.save('/content/drive/MyDrive/New_MF_data/New_Class/my_model_5Hz_1000.h5')  # This will save in the TensorFlow SavedModel format by default

# Load the saved model

In [ ]:
from tensorflow.keras.models import load_model

# Load the model previously saved in HDF5 format
model_h5 = load_model('/content/drive/MyDrive/New_MF_data/New_Class/my_model_5Hz_1000.h5')

# Training and validation accuracy and loss curves for all cross validation technique

In [ ]:
# Function to plot the history of all folds
def plot_history(histories):
    for i, history in enumerate(histories):
        plt.figure(figsize=(14, 5))

        # Plot accuracy
        plt.subplot(1, 2, 1)
        plt.plot(history.history['accuracy'], label='Train Accuracy')
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.title(f'Fold {i+1} Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()

        # Plot loss
        plt.subplot(1, 2, 2)
        plt.plot(history.history['loss'], label='Train Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.title(f'Fold {i+1} Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()

        plt.show()

plot_history(results)

# Confusion matrix, accuracy and prediction of the model

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np
import itertools
def plot_confusion_matrix(cm, class_labels):
    plt.figure(figsize=(10, 7))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
    tick_marks = np.arange(len(class_labels))
    plt.xticks(tick_marks, class_labels, rotation=45)
    plt.yticks(tick_marks, class_labels)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, accuracy_score

# Setup the ImageDataGenerator
test_datagen = ImageDataGenerator(rescale=1./255)

# Create a data generator for the test set
test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/New_MF_data/New_Class/5HZ_Dataset_DNN/test',  # Path to test data
    target_size=(224, 224),  # Assuming using 224x224 for ResNet input
    batch_size=32,  # Batch size
    class_mode='categorical',  # Type of problem
    shuffle=False  # No need to shuffle for testing
)

# Modify the predict_and_evaluate function to use the correct variable names
def predict_and_evaluate(model, generator):
    generator.reset()
    pred_indices = model.predict(generator, steps=generator.samples // generator.batch_size + 1)
    predictions = np.argmax(pred_indices, axis=1)

    true_classes = generator.classes
    class_labels = list(generator.class_indices.keys())

    cm = confusion_matrix(true_classes, predictions)
    accuracy = accuracy_score(true_classes, predictions)  # Calculate the accuracy
    return cm, class_labels, accuracy

# Now, call the function with the correct generator
conf_matrix, labels, test_accuracy = predict_and_evaluate(model, test_generator)
print("Test Accuracy:", test_accuracy*100)
plot_confusion_matrix(conf_matrix, labels)
